# Riverscapes Exploration Notebook

This notebook is configured to allow interactive analysis of Riverscapes data.
It sets up the python path to include the `src` directory, enabling imports from `reports` and `util`.


In [1]:
import sys
import os
from pathlib import Path

# Add the src directory to the path so we can import from reports and util
# This assumes the notebook is running from scripts/exploration/
# Adjust relative path if moved
current_dir = Path.cwd().resolve()
# We want to go up two levels from scripts/exploration to root
project_root = current_dir.parent.parent

src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print(f"Project root added to path: {project_root}")


Project root added to path: C:\nardata\localcode\rs-reports-gen


In [2]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
import matplotlib.pyplot as plt

# Import from src
# These imports will work once the path is set up above
from util.athena import query_to_dataframe, get_field_metadata
# this is only needed if the util code has changed in middle of our session
# import importlib
# import util.pandas.pandas_utilities
# importlib.reload(util.pandas.pandas_utilities)
from util.pandas.pandas_utilities import pprint_df_meta
from util.pandas import RSFieldMeta

# Set pandas options for better display
pd.set_option('display.max_columns', None)


In [3]:
def get_data()->pd.DataFrame:
    # Define the query (from notebook.py)
    qry = """
    SELECT r.seg_distance, r.fcode, rm.* 
    FROM rsdynamics r 
    join rsdynamics_metrics rm on (r.rd_project_id = rm.rd_project_id and r.dgo_id = rm.dgo_id)
    where r.huc='364256' and level_path = '364256000000001'
    """

    # alternate query - when want to get geometry and other data from the rsdynamics as well as the metrics
    qry = """
    SELECT
        r.rd_project_id,
        r.dgo_id,  -- Use ID from 'r' for grouping
        -- Optimization: Don't group by Geometry. Use ARBITRARY() or any_value to pass it through.
        any_value(r.huc) AS huc,
        any_value(r.dgo_geom) AS dgo_geom,
        any_value(r.seg_distance) AS seg_distance,

        -- Collapse the 30 metric rows into one list of structs
    ARRAY_AGG(
        CAST(
            ROW(
                m.landcover,
                m.epoch_length,
                m.epoch_name,
                m.confidence,
                m.area,
                m.areapc,
                m.width,
                m.widthpc
            ) AS ROW(
                landcover VARCHAR,
                epoch_length VARCHAR,
                epoch_name VARCHAR,
                confidence VARCHAR,
                area DOUBLE,
                areapc DOUBLE,
                width DOUBLE,
                widthpc DOUBLE
            )
        )
    ) AS metrics_list

    FROM rsdynamics r
    JOIN rsdynamics_metrics m
        ON (r.rd_project_id = m.rd_project_id AND r.dgo_id = m.dgo_id)
    WHERE r.huc = '593468'
    GROUP BY
        r.huc,
        r.rd_project_id,
        r.dgo_id
    """

    # Load the dataframe
    print("Querying Athena...")
    # Note: Ensure you have AWS credentials configured in your environment
    try:
        df_raw = query_to_dataframe(qry, "dynamicsmetrics")
        print(f"Dataframe loaded with shape: {df_raw.shape}")
        display(df_raw.head())
        return df_raw
    except Exception as e:
        print(f"Error executing query: {e}")
        print("Ensure AWS credentials are set and 'util.athena' is reachable.")

df_raw = get_data()



Querying Athena...
[DEBUG] [Athena unload query to DF] Query dynamicsmetrics:
              
                  SELECT
                      r.rd_project_id,
                      r.dgo_id,  -- Use ID from 'r' for grouping
                      -- Optimization: Don't group by Geometry. Use ARBITRARY() or any_value to pass it through.
                      any_value(r.huc) AS huc,
                      any_value(r.dgo_geom) AS dgo_geom,
                      any_value(r.seg_distance) AS seg_distance,
              
                      -- Collapse the 30 metric rows into one list of structs
                  ARRAY_AGG(
                      CAST(
                          ROW(
                              m.landcover,
                              m.epoch_length,
                              m.epoch_name,
                              m.confidence,
                              m.area,
                              m.areapc,
                              m.width,
                     

,rd_project_id,dgo_id,huc,dgo_geom,seg_distance,metrics_list
0,907f9a2f-a594-433f-a248-970798998e0b,1,593468,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...,50,"[{'landcover': 'wet', 'epoch_length': '5', 'ep..."
1,907f9a2f-a594-433f-a248-970798998e0b,9,593468,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...,50,"[{'landcover': 'wet', 'epoch_length': '5', 'ep..."
2,907f9a2f-a594-433f-a248-970798998e0b,27,593468,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...,3250,"[{'landcover': 'wet', 'epoch_length': '5', 'ep..."
3,907f9a2f-a594-433f-a248-970798998e0b,28,593468,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...,2950,"[{'landcover': 'wet', 'epoch_length': '5', 'ep..."
4,907f9a2f-a594-433f-a248-970798998e0b,30,593468,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...,3050,"[{'landcover': 'wet', 'epoch_length': '5', 'ep..."


In [4]:
def restructure_dataframe(df_raw:pd.DataFrame)->tuple[gpd.GeoDataFrame, pd.DataFrame]:
    # 1. Convert to category and SET THE INDEX IMMEDIATELY
    # This establishes the "rd_project_id + dgo_id" as the unique identifier
    for col in ['rd_project_id', 'huc']:
        df_raw[col] = df_raw[col].astype('category')
    df_raw = df_raw.set_index(['rd_project_id', 'dgo_id'])

    # ---------------------------------------------------------
    # 2. CREATE GEO DATAFRAME (The 1 side of 1:N)
    # ---------------------------------------------------------
    # Since the index is already set, we just grab the columns we want.
    # The index (rd_project_id, dgo_id) stays attached automatically.
    df_geo_data = df_raw[['huc', 'dgo_geom','seg_distance']].copy()

    # Convert to GeoDataFrame
    # 1. Fast Vectorized Conversion
    # This is much faster than .apply(wkb.loads)
    df_geo_data['dgo_geom'] = gpd.GeoSeries.from_wkb(df_geo_data['dgo_geom'])
    # although it is standard to name the geom column geometry, it isn't required, and since we deal with many geoms I like to keep it more specific

    # 2. "Cast" to GeoDataFrame (Overwrite the variable)
    #    This consumes the old DataFrame and returns the GeoDataFrame wrapper.
    #    Pandas is smart enough to pass the data by reference here (it's a view),
    #    so no deep copy of the data occurs.
    df_geo_data = gpd.GeoDataFrame(df_geo_data, geometry='dgo_geom')

    # Now df_geo_data has all the spatial powers (crs, plot, etc.)
    # df_geo_data.set_crs(epsg=4326, inplace=True)

    # A. Explode: Turn 1 row with a list of 30 items into 30 rows
    exploded_series = df_raw['metrics_list'].explode()

    # B. Normalize: Turn the dictionary/struct column into separate columns
    # AWS Wrangler usually returns ROW types as Python dictionaries
    df_metrics = pd.DataFrame(
        exploded_series.tolist(),
        index=exploded_series.index
    )
    return (df_geo_data, df_metrics)

In [5]:
df_geo_data, df_metrics = restructure_dataframe(df_raw)

In [9]:
bar = '='*12
print(f'{bar}> df_raw <{bar}')
print(df_raw.info())
print(f'{bar}> df_metrics <{bar}')
print(df_metrics.info())
print(f'{bar}> df_geo_data <{bar}')
print(df_geo_data.info())

============> df_raw <============
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20220 entries, 0 to 20219
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   rd_project_id  20220 non-null  category
 1   dgo_id         20220 non-null  Int64   
 2   huc            20220 non-null  category
 3   dgo_geom       20220 non-null  object  
 4   seg_distance   19661 non-null  Int64   
 5   metrics_list   20220 non-null  object  
dtypes: Int64(2), category(2), object(2)
memory usage: 711.2+ KB
None
============> df_metrics <============
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 566160 entries, ('907f9a2f-a594-433f-a248-970798998e0b', np.int64(1)) to ('907f9a2f-a594-433f-a248-970798998e0b', np.int64(28237))
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   landcover     566160 non-null  object 
 1   epoch_length  566160 non-null  object 

In [10]:
# Interactive Visualization
if 'df_geo_data' in locals() and not df_geo_data.empty:
    # Example: Histogram of segment distance
    if 'seg_distance' in df_geo_data.columns:
        fig = px.histogram(df_geo_data, x="seg_distance", title="Distribution of Segment Distances")
        fig.show()
    
    # You can add more plots here using px.scatter, px.line, etc.
else:
    print("Dataframe not available for plotting.")


In [11]:
# Convert columns to categorical
cols_to_convert = ['confidence', 'landcover', 'fcode', 'epoch_name','epoch_length']
for col in cols_to_convert:
    if col in df_metrics.columns:
        df_metrics[col] = df_metrics[col].astype('category')
df_metrics.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 566160 entries, ('907f9a2f-a594-433f-a248-970798998e0b', np.int64(1)) to ('907f9a2f-a594-433f-a248-970798998e0b', np.int64(28237))
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   landcover     566160 non-null  category
 1   epoch_length  566160 non-null  category
 2   epoch_name    566160 non-null  category
 3   confidence    566160 non-null  category
 4   area          566160 non-null  float64 
 5   areapc        51635 non-null   float64 
 6   width         553252 non-null  float64 
 7   widthpc       51055 non-null   float64 
dtypes: category(4), float64(4)
memory usage: 21.7 MB


In [33]:
dynmetrics = df_metrics.copy()

df_30yr_area = dynmetrics[
        (dynmetrics['epoch_name'] == '1989_2024') &
        (dynmetrics['confidence'] == '95')
    ]
print(df_30yr_area.describe())
# print(df_30yr_area['area'].sort_values())
df_30yr_area['area_bin'] = pd.cut(df_30yr_area['area'],10)
print(df_30yr_area)

               area       areapc         width      widthpc
count  4.044000e+04  1293.000000  39518.000000  1280.000000
mean   1.839854e+03    50.507874      2.441555    50.102106
std    4.243591e+04    30.431497     29.355140    30.278167
min    0.000000e+00     0.044079      0.000000     0.044079
25%    0.000000e+00    24.948973      0.000000    24.379057
50%    0.000000e+00    49.281188      0.000000    48.734598
75%    0.000000e+00    74.482546      0.000000    73.257867
max    2.470834e+06   100.000000   1862.692475   100.000000
                                            landcover epoch_length epoch_name  \
rd_project_id                        dgo_id                                     
907f9a2f-a594-433f-a248-970798998e0b 1            wet           30  1989_2024   
                                     1         active           30  1989_2024   
                                     9            wet           30  1989_2024   
                                     9         active  

C:\Users\narlorin\AppData\Local\Temp\ipykernel_25964\3196370444.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [36]:
# pandas.cut example: Binning data into intervals
# documentation: https://pandas.pydata.org/docs/reference/api/pandas.cut.html

# 1. Automatic equal-width bins: Split data into 5 bins
print("--- Equal Width Bins ---")
area_bins_equal = pd.cut(df_30yr_area['area'], bins=5)
print(area_bins_equal.value_counts().sort_index())

# 2. Custom bin edges
# e.g., 0-100, 100-1000, 1000+
print("\n--- Custom Bins ---")
custom_bins = [0, 1000, 5000, float('inf')]
labels = ['Small (<1k)', 'Medium (1k-5k)', 'Large (>5k)']

# 3. Add as column to allow multi-variable grouping
df_30yr_area['area_category'] = pd.cut(df_30yr_area['area'], bins=custom_bins, labels=labels)

# Group by both Landcover AND Area Category
# observed=False ensures we see all categories even if count is 0
grouped_counts = df_30yr_area.groupby(['landcover', 'area_category'], observed=False).size()
print(grouped_counts)
print("\n--- Counts by Landcover & Size (Unstacked) ---")
print(grouped_counts.unstack()) 

print("\n--- Counts by Landcover & Size (Table) ---")
# Another way to view it using crosstab (frequency table)
print(pd.crosstab(df_30yr_area['landcover'], df_30yr_area['area_category']))

--- Equal Width Bins ---
area
(-2470.834, 494166.837]       40401
(494166.837, 988333.673]         19
(988333.673, 1482500.51]          9
(1482500.51, 1976667.347]         6
(1976667.347, 2470834.184]        5
Name: count, dtype: int64

--- Custom Bins ---
landcover  area_category 
active     Small (<1k)       360
           Medium (1k-5k)    436
           Large (>5k)       497
wet        Small (<1k)         0
           Medium (1k-5k)      0
           Large (>5k)         0
dtype: int64

--- Counts by Landcover & Size (Unstacked) ---
area_category  Small (<1k)  Medium (1k-5k)  Large (>5k)
landcover                                              
active                 360             436          497
wet                      0               0            0

--- Counts by Landcover & Size (Table) ---
area_category  Small (<1k)  Medium (1k-5k)  Large (>5k)
landcover                                              
active                 360             436          497


C:\Users\narlorin\AppData\Local\Temp\ipykernel_25964\2168633657.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [37]:
from util.figures import bar_group_x_by_y

# To use bar_group_x_by_y we need to pass:
# 1. The DataFrame (must contain the columns we are grouping/summing)
# 2. 'total_col': The numerical column to sum (e.g., 'area')
#    NOTE: grouped_counts is a Series of counts, so we can't use it directly if we want SUM of area.
#    However, if we want to plot the Area Sums:
# 3. 'group_by_cols': A list where:
#    - [0] = Y-axis category (e.g., 'area_category')
#    - [1] = Color category (e.g., 'landcover')

# FIX for "Object of type Interval is not JSON serializable"
# Convert the interval column to string before passing to plotly
if 'area_category' in df_30yr_area.columns:
    df_30yr_area['area_category'] = df_30yr_area['area_category'].astype(str)

# Example 1: Plotting Sum of Area
fig_area = bar_group_x_by_y(
    df_30yr_area, 
    total_col='area', 
    group_by_cols=['area_category', 'landcover'],
    fig_params={'title': 'Total Area by Size Category and Landcover'}
)
fig_area.show()

# Example 2: If you specifically wanted to plot the *Count* of records (the histogram idea)
# We need a column that represents "1" to sum up, or use a dummy column.
df_30yr_area['record_count'] = 1
fig_count = bar_group_x_by_y(
    df_30yr_area, 
    total_col='record_count', 
    group_by_cols=['area_category', 'landcover'],
    fig_params={
        'title': 'Number of Polygons by Size Category and Landcover',
        'xaxis_title': 'Count'
    }
)
fig_count.show()

[DEBUG] [RSFieldMeta] Set default unit system to SI
[WARNING] [RSFieldMeta] No metadata set. Remember to instantiate the RSFieldMeta using RSFieldMeta().df = meta_df
[WARNING] [RSFieldMeta] No metadata set. Remember to instantiate the RSFieldMeta using RSFieldMeta().df = meta_df
[WARNING] [RSFieldMeta] No metadata set. Remember to instantiate the RSFieldMeta using RSFieldMeta().df = meta_df
[WARNING] [RSFieldMeta] No metadata set. Remember to instantiate the RSFieldMeta using RSFieldMeta().df = meta_df
[WARNING] [RSFieldMeta] No metadata set. Remember to instantiate the RSFieldMeta using RSFieldMeta().df = meta_df
[WARNING] [RSFieldMeta] No metadata set. Remember to instantiate the RSFieldMeta using RSFieldMeta().df = meta_df
[INFO] [RSFieldMeta] Applying Units


RuntimeError: No metadata set. You need to instantiate RSFieldMeta and set the .meta property first.

In [10]:
# Filter data where epoch_length is 5 (including all confidence levels)
filtered_df = df_metrics[(df_metrics['epoch_length'] == '5')].copy()
# however when you do only that the epoch_length category still includes the epoch_length 30

# 1. Isolate the Series for the 5-year epochs
subset_series = df_metrics.loc[df_metrics['epoch_length'] == '5', 'epoch_name']

# 2. Remove unused categories from the SERIES, then grab the remaining categories
#    This returns a Pandas Index, which set_categories() accepts happily.
valid_5yr_epochs = subset_series.cat.remove_unused_categories().cat.categories

# 3. Apply to your filtered dataframe
filtered_df['epoch_name'] = filtered_df['epoch_name'].cat.set_categories(valid_5yr_epochs)


# 4. Now run your GroupBy
# Summarize sum of area by landcover, epoch_name, and confidence
#    observed=False will now show all valid_5yr_epochs (even empty ones),
#    but will NOT show the 30-year epoch.
summary_df = filtered_df.groupby(
    ['landcover', 'epoch_name', 'confidence'], 
    observed=False
)['area'].sum().reset_index()

# Sort by epoch_name
summary_df = summary_df.sort_values('epoch_name')

In [11]:
print(valid_5yr_epochs)
# print(filtered_df.shape)
# print(summary_df.shape)

Index(['1989_1994', '1999_2004', '2004_2009', '2009_2014', '2014_2019',
       '2019_2024'],
      dtype='object')


In [12]:
# print(filtered_df.groupby(['epoch_name','epoch_length'],observed=True).count())
print(summary_df)

   landcover epoch_name confidence          area
0     active  1989_1994         68  1.979401e+08
1     active  1989_1994         95  1.979401e+08
13       wet  1989_1994         95  7.910859e+07
12       wet  1989_1994         68  7.910859e+07
2     active  1999_2004         68  1.867352e+08
3     active  1999_2004         95  1.282326e+08
15       wet  1999_2004         95  4.664323e+06
14       wet  1999_2004         68  6.377014e+07
4     active  2004_2009         68  1.649480e+08
5     active  2004_2009         95  1.095055e+08
17       wet  2004_2009         95  2.340475e+06
16       wet  2004_2009         68  5.838352e+07
19       wet  2009_2014         95  6.418183e+05
18       wet  2009_2014         68  3.635285e+07
7     active  2009_2014         95  9.302774e+07
6     active  2009_2014         68  1.676576e+08
21       wet  2014_2019         95  6.384928e+04
8     active  2014_2019         68  1.548811e+08
20       wet  2014_2019         68  2.384704e+07
9     active  2014_2

In [13]:
import plotly.express as px
# Create a line chart with different styles for confidence
fig = px.line(summary_df, x='epoch_name', y='area', color='landcover', line_dash='confidence',
              title='Area by Landcover over Time (Epoch Length 5)',
              line_dash_map={'95': 'solid', '68': 'dash'})
fig.show()

In [14]:
# Loop through metrics and create plots
metrics = ['area', 'areapc', 'width', 'widthpc']
for metric in metrics:
    if metric in df_metrics.columns:
        # Group by landcover, epoch_name, confidence
        metric_summary = filtered_df.groupby(['landcover', 'epoch_name', 'confidence'], observed=False)[metric].sum().reset_index()
        metric_summary = metric_summary.sort_values('epoch_name')
        
        # Create line chart
        fig = px.line(metric_summary, x='epoch_name', y=metric, color='landcover', line_dash='confidence',
                      title=f'{metric.capitalize()} by Landcover over Time (Epoch Length 5)',
                      line_dash_map={'95': 'solid', '68': 'dash'})
        fig.show()

In [15]:
pprint_df_meta(df_metrics)

DataFrame: Unnamed
------------------------------------------------------------------------------------------------------------------------
Shape (rows, cols): (566160, 8)

[DEBUG] [RSFieldMeta] Set default unit system to SI
[WARNING] [RSFieldMeta] No metadata set. Remember to instantiate the RSFieldMeta using RSFieldMeta().df = meta_df
[WARNING] [RSFieldMeta] No metadata set. Remember to instantiate the RSFieldMeta using RSFieldMeta().df = meta_df
[WARNING] [RSFieldMeta] No metadata set. Remember to instantiate the RSFieldMeta using RSFieldMeta().df = meta_df
[WARNING] [RSFieldMeta] No metadata set. Remember to instantiate the RSFieldMeta using RSFieldMeta().df = meta_df
[WARNING] [RSFieldMeta] No metadata set. Remember to instantiate the RSFieldMeta using RSFieldMeta().df = meta_df
[WARNING] [RSFieldMeta] No metadata set. Remember to instantiate the RSFieldMeta using RSFieldMeta().df = meta_df
[WARNING] [RSFieldMeta] No metadata set. Remember to instantiate the RSFieldMeta using RSFi

In [16]:
# Generate Report Artifacts / Metadata Exploration
# This section demonstrates using other parts of src/reports or util

if 'df_metrics' in locals():
    # Example: Check field metadata using util function
    try:
        _FIELD_META = RSFieldMeta()  # Instantiate the Borg singleton. We can reference it with this object or RSFieldMeta()
        metadf = get_field_metadata(authority='data-exchange-scripts', authority_name='rsdynamics_to_athena', layer_id=["rsdynamics","rsdynamics_metrics"])
        _FIELD_META.field_meta = metadf
        print("\nField Metadata:")
        # Display first few keys
        keys = list(metadf.keys())[:5]
        for k in keys:
            print(f"{k}: {metadf[k]}")
    except Exception as e:
        print(f"Could not retrieve metadata: {e}")


[INFO] [Get metadata] Getting metadata from Athena
[DEBUG] [Athena unload query to DF] Query :
              
              SELECT layer_id, layer_name, name, friendly_name, data_unit, description, theme, dtype
              FROM layer_definitions_latest
              WHERE authority = 'data-exchange-scripts'  AND authority_name IN ('rsdynamics_to_athena')  AND layer_id IN ('rsdynamics', 'rsdynamics_metrics') 
              
[DEBUG] [Athena unload query to DF] Query  to dataframe completed.
[INFO] [RSFieldMeta] Setting metadata for the first time.

Field Metadata:
layer_id: 0             rsdynamics
1             rsdynamics
2             rsdynamics
3             rsdynamics
4             rsdynamics
5             rsdynamics
6             rsdynamics
7             rsdynamics
8             rsdynamics
9             rsdynamics
10            rsdynamics
11            rsdynamics
12            rsdynamics
13            rsdynamics
14            rsdynamics
15            rsdynamics
16            rsdyn

In [17]:
print(metadf.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   layer_id       33 non-null     string
 1   layer_name     33 non-null     string
 2   name           33 non-null     string
 3   friendly_name  33 non-null     string
 4   data_unit      33 non-null     string
 5   description    33 non-null     string
 6   theme          33 non-null     string
 7   dtype          33 non-null     string
dtypes: string(8)
memory usage: 2.2 KB
None


In [24]:
from util.figures import (bar_group_x_by_y)
dynmetrics = df_metrics.copy()
df_30yr_area = dynmetrics[
    (dynmetrics['epoch_name'] == '1989_2024') &
    (dynmetrics['confidence'] == '68')
]
print(df_30yr_area.head())
fig = bar_group_x_by_y(df_30yr_area, 'area', ['landcover'])
fig.show()

                                            landcover epoch_length epoch_name  \
rd_project_id                        dgo_id                                     
907f9a2f-a594-433f-a248-970798998e0b 1            wet           30  1989_2024   
                                     1         active           30  1989_2024   
                                     9            wet           30  1989_2024   
                                     9         active           30  1989_2024   
                                     27           wet           30  1989_2024   

                                            confidence        area     areapc  \
rd_project_id                        dgo_id                                     
907f9a2f-a594-433f-a248-970798998e0b 1              68    0.000000        NaN   
                                     1              68  665.096685  54.695451   
                                     9              68    0.000000        NaN   
                           